In [1]:
'''
2022.11.15，复现HAN(Heterogeneous Graph Attention Network)
dependencies:
    tensorflow-2.11.0
    numpy-1.22.0
    networkx-2.6.3
    scipy-1.7.3
'''

'\n2022.11.15，复现HAN(Heterogeneous Graph Attention Network)\ndependencies:\n    tensorflow-2.11.0\n    numpy-1.22.0\n    networkx-2.6.3\n    scipy-1.7.3\n'

In [2]:
import numpy as np
import pandas as pd

import os
project_path = os.getcwd()

In [3]:
project_path

'D:\\PycharmProjects\\GNN_Event_Detection_models\\Re-HAN Models'

# utils

## data process

In [4]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys

In [5]:
"""
 Prepare adjacency matrix by expanding up to a given neighbourhood.
 This will insert loops on every node.
 Finally, the matrix is converted to bias vectors.
 Expected shape: [graph, nodes, nodes]
"""

'\n Prepare adjacency matrix by expanding up to a given neighbourhood.\n This will insert loops on every node.\n Finally, the matrix is converted to bias vectors.\n Expected shape: [graph, nodes, nodes]\n'

### adj_to_bias

In [6]:
np.eye(3,4)[0] + np.eye(3,4)

array([[2., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 1., 0.]])

In [7]:
def adj_to_bias(adj, sizes, nhood=1):  # 邻接矩阵adjacency matrix
    nb_graphs = adj.shape[0]  # 行,即num_nodes
    mt = np.empty(adj.shape)  # 根据给定的维度和数值类型，返回一个新的ndarray数组，其元素不进行初始化
    for g in range(nb_graphs):
        mt[g] = np.eye(adj.shape[1])  # 返回一个二维的ndarray数组
        for _ in range(nhood):
            mt[g] = np.matmul(mt[g], (adj[g] + np.eye(adj.shape[1])))  # 相乘
        for i in range(sizes[g]):  # 这个应该可以简化，直接对整个数组元素做操作！！！
            for j in range(sizes[g]):
                if mt[g][i][j] > 0.0:
                    mt[g][i][j] = 1.0
    return -1e9 * (1.0 - mt)  # 科学计数法，2.5 x 10^(-27)表示为：2.5e-27

###  parse_index_file

In [8]:
# load file
def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

### sample_mask

In [9]:
# 生成掩码bool数组
def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)  # 生成全是0的数组
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

### sparse_to_tuple

In [10]:
'''转换为稀疏矩阵tuple'''
def to_tuple(mx):
    if not sp.isspmatrix_coo(mx):
        mx = mx.tocoo()
    coords = np.vstack((mx.row, mx.col)).transpose()
    values = mx.data
    shape = mx.shape
    return coords, values, shape

In [11]:
def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""

    if isinstance(sparse_mx, list):
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx

### standardize_data

In [12]:
def standardize_data(f, train_mask):
    """Standardize feature matrix and convert to tuple representation"""
    # standardize data
    f = f.todense()  # 将稀疏矩阵转回numpy矩阵
    mu = f[train_mask == True, :].mean(axis=0)  # 按行求平均
    sigma = f[train_mask == True, :].std(axis=0)
    f = f[:, np.squeeze(np.array(sigma > 0))]  # sigma>0 get bool array
    mu = f[train_mask == True, :].mean(axis=0)
    sigma = f[train_mask == True, :].std(axis=0)
    f = (f - mu) / sigma
    return f

### preprocess_features

In [13]:
def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()  # power数组元素求n次方，flatten是降到一维
    r_inv[np.isinf(r_inv)] = 0.  # isinf判断是否为无穷
    r_mat_inv = sp.diags(r_inv)  # 从对角线构造一个稀疏矩阵。
    features = r_mat_inv.dot(features)  # dot矩阵乘法
    return features.todense(), sparse_to_tuple(features)  # todense()转换成密集矩阵numpy.matrix

### normalize_adj

In [14]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix. 对称归一化邻接矩阵"""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

### preprocess_adj

In [15]:
def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))  # 对角线为1的矩阵
    return sparse_to_tuple(adj_normalized)

## layers

In [16]:
import numpy as np
import tensorflow as tf
import tf_slim

### attn_head

In [17]:
def attn_head(features, out_sz, bias_mat, activation, in_drop=0.0, coef_drop=0.0, residual=False,
              return_coef=False):
    """[summary]
    multi-head attention计算
    [description]
    # forward；model = HeteGAT_multi
    attns.append(layers.attn_head(features,            # list:3, tensor（1， 3025， 1870）
                                bias_mat=bias_mat,     # list:2, tensor(1, 3025, 3025)
                                out_sz=hid_units[0],   # hid_units:[8]，卷积核的个数
                                activation=activation, # nonlinearity:tf.nn.elu
                                in_drop=ffd_drop,      # tensor, ()
                                coef_drop=attn_drop,   # tensor, ()
                                residual=False))
    Arguments:
        features {[type]} -- shape=(batch_size, nb_nodes, fea_size))
    """
    with tf.name_scope('my_attn'):  # 定义一个上下文管理器
        if in_drop != 0.0:
            features = tf.nn.dropout(features, 1.0 - in_drop)  # 以rate置0
        features_fts = tf.compat.v1.layers.conv1d(features, out_sz, 1, use_bias=False)  # 一维卷积操作, out: (1, 3025, 8)
        
        f_1 = tf.compat.v1.layers.conv1d(features_fts, 1, 1)  # (1, 3025, 1)
        f_2 = tf.compat.v1.layers.conv1d(features_fts, 1, 1)  # (1, 3025, 1)
        
        logits = f_1 + tf.transpose(f_2, [0, 2, 1])  # 转置         # (1, 3025, 3025)
        coefs = tf.nn.softmax(tf.nn.leaky_relu(logits) + bias_mat)  # (1, 3025, 3025)

        if coef_drop != 0.0:
            coefs = tf.nn.dropout(coefs, 1.0 - coef_drop)           
        if in_drop != 0.0:
            features_fts = tf.nn.dropout(features_fts, 1.0 - in_drop)

        vals = tf.matmul(coefs, features_fts)                       # (1, 3025, 8)
        ret = tf_slim.bias_add(vals)  # 将bias向量加到value矩阵上      # (1. 3025， 8)

        # residual connection 残差连接
        if residual:
            if features.shape[-1] != ret.shape[-1]:
                ret = ret + tf.compat.v1.layers.conv1d(features, ret.shape[-1], 1)  # activation
            else:
                features_fts = ret + features
        if return_coef:
            return activation(ret), coefs
        else:
            return activation(ret)  # activation

### attn_head_const_1

In [18]:
def attn_head_const_1(seq, out_sz, bias_mat, activation, in_drop=0.0, coef_drop=0.0, residual=False):
    """[summary]
    [description]
    """
    adj_mat = 1.0 - bias_mat / -1e9
    with tf.name_scope('my_attn'):
        if in_drop != 0.0:
            seq = tf.nn.dropout(seq, 1.0 - in_drop)
        seq_fts = tf.compat.v1.layers.conv1d(seq, out_sz, 1, use_bias=False)
        

        logits = adj_mat 
        coefs = tf.nn.softmax(tf.nn.leaky_relu(logits) + bias_mat)

        if coef_drop != 0.0:
            coefs = tf.nn.dropout(coefs, 1.0 - coef_drop)
        if in_drop != 0.0:
            seq_fts = tf.nn.dropout(seq_fts, 1.0 - in_drop)

        vals = tf.matmul(coefs, seq_fts)
        ret = tf_slim.bias_add(vals)

        # residual connection
        if residual:
            if seq.shape[-1] != ret.shape[-1]:
                ret = ret + tf.compat.v1.layers.conv1d(seq, ret.shape[-1], 1)  # activation
            else:
                seq_fts = ret + seq

        return activation(ret)  # activation

### sp_attn_head

In [19]:
def sp_attn_head(seq, out_sz, adj_mat, activation, nb_nodes, in_drop=0.0, coef_drop=0.0, residual=False):
    with tf.name_scope('sp_attn'):
        if in_drop != 0.0:
            seq = tf.nn.dropout(seq, 1.0 - in_drop)

        seq_fts = tf.compat.v1.layers.conv1d(seq, out_sz, 1, use_bias=False)

        # simplest self-attention possible
        f_1 = tf.compat.v1.layers.conv1d(seq_fts, 1, 1)
        f_2 = tf.compat.v1.layers.conv1d(seq_fts, 1, 1)
        logits = tf.sparse_add(adj_mat * f_1, adj_mat *
                               tf.transpose(f_2, [0, 2, 1]))
        lrelu = tf.SparseTensor(indices=logits.indices,
                                values=tf.nn.leaky_relu(logits.values),
                                dense_shape=logits.dense_shape)
        coefs = tf.sparse_softmax(lrelu)  # 将softmax应用于批量的N维SparseTensor

        if coef_drop != 0.0:
            coefs = tf.SparseTensor(indices=coefs.indices,
                                    values=tf.nn.dropout(
                                        coefs.values, 1.0 - coef_drop),
                                    dense_shape=coefs.dense_shape)
        if in_drop != 0.0:
            seq_fts = tf.nn.dropout(seq_fts, 1.0 - in_drop)

        # As tf.sparse_tensor_dense_matmul expects its arguments to have rank-2,
        # here we make an assumption that our input is of batch size 1, and reshape appropriately.
        # The method will fail in all other cases!
        coefs = tf.sparse_reshape(coefs, [nb_nodes, nb_nodes])
        seq_fts = tf.squeeze(seq_fts)  
        vals = tf.sparse_tensor_dense_matmul(coefs, seq_fts)  # SparseTensor稀疏矩阵乘法
        vals = tf.expand_dims(vals, axis=0)  # 在0处扩展维度1
        vals.set_shape([1, nb_nodes, out_sz])
        ret = tf_slim.bias_add(vals)

        # residual connection
        if residual:
            if seq.shape[-1] != ret.shape[-1]:
                ret = ret + tf.compat.v1.layers.conv1d(seq, ret.shape[-1], 1)  # activation
            else:
                seq_fts = ret + seq

        return activation(ret)  # activation

### SimpleAttLayer

In [20]:
# final_embed, att_val = layers.SimpleAttLayer(multi_embed, mp_att_size,
#                                                      time_major=False,
#                                                      return_alphas=True)
def SimpleAttLayer(inputs, attention_size, time_major=False, return_alphas=False):
    '''
    inputs: tensor, (3025, 2, 64)
    attention_size: 128
    '''
    if isinstance(inputs, tuple):
        # In case of Bi-RNN, concatenate the forward and the backward RNN outputs.
        inputs = tf.concat(inputs, 2)  # 表示在shape第2个维度上拼接

    if time_major:
        # (T,B,D) => (B,T,D)
        inputs = tf.array_ops.transpose(inputs, [1, 0, 2])  #

    hidden_size = inputs.shape[2]  # D value - hidden size of the RNN layer

    # Trainable parameters
    w_omega = tf.Variable(tf.compat.v1.random_normal([hidden_size, attention_size], stddev=0.1))  # (64, 128)
    b_omega = tf.Variable(tf.compat.v1.random_normal([attention_size], stddev=0.1))               # (128, )
    u_omega = tf.Variable(tf.compat.v1.random_normal([attention_size], stddev=0.1))               # (128, )

    with tf.name_scope('v'):
        # Applying fully connected layer with non-linear activation to each of the B*T timestamps;
        #  the shape of `v` is (B,T,D)*(D,A)=(B,T,A), where A=attention_size
        v = tf.tanh(tf.tensordot(inputs, w_omega, axes=1) + b_omega)   # (3025, 2, 128)

    # For each of the timestamps its vector of size A from `v` is reduced with `u` vector
    vu = tf.tensordot(v, u_omega, axes=1, name='vu')  # (B,T) shape   tensor, (3025, 2)
    alphas = tf.nn.softmax(vu, name='alphas')         # (B,T) shape   tensor, (3025, 2)

    # Output of (Bi-)RNN is reduced with attention vector; the result has (B,D) shape
    output = tf.reduce_sum(inputs * tf.expand_dims(alphas, -1), 1)  # (3025, 2, 64) * (3025, 2, 1) = (3025, 2, 64) -> (3025, 2)

    if not return_alphas:
        return output
    else:
        return output, alphas  # attention输出、softmax概率

# clustering

In [21]:
import os
import numpy as np

#import matplotlib
#matplotlib.use('Agg')
#import matplotlib.pyplot as plt

import pickle  # 把训练好的模型存储起来

from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score,adjusted_mutual_info_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import roc_curve, f1_score
from sklearn import manifold  # 一种非线性降维的手段
from sklearn.model_selection import train_test_split

## KNN_model

In [22]:
def my_KNN(x, y, k=5, split_list=[0.2, 0.4, 0.6, 0.8], time=10, show_train=True, shuffle=True):
    x = np.array(x)
    x = np.squeeze(x)
    y = np.array(y)
    if len(y.shape) > 1:
        y = np.argmax(y, axis=1)
    for split in split_list:
        ss = split
        split = int(x.shape[0] * split)
        micro_list = []
        macro_list = []
        if time:
            for i in range(time):
                if shuffle:
                    permutation = np.random.permutation(x.shape[0])  # 生成一个随机打散的序列。
                    x = x[permutation, :]
                    y = y[permutation]
                # x_true = np.array(x_true)
                train_x = x[:split, :]
                test_x = x[split:, :]

                train_y = y[:split]
                test_y = y[split:]

                estimator = KNeighborsClassifier(n_neighbors=k)
                estimator.fit(train_x, train_y)
                y_pred = estimator.predict(test_x)
                f1_macro = f1_score(test_y, y_pred, average='macro')
                f1_micro = f1_score(test_y, y_pred, average='micro')
                macro_list.append(f1_macro)
                micro_list.append(f1_micro)
            print('KNN({}avg, split:{}, k={}) f1_macro: {:.4f}, f1_micro: {:.4f}'.format(
                time, ss, k, sum(macro_list) / len(macro_list), sum(micro_list) / len(micro_list)))

## kmeans_model

In [23]:
def my_Kmeans(x, y, k=4, time=10, return_NMI=False):

    x = np.array(x)
    x = np.squeeze(x)
    y = np.array(y)

    if len(y.shape) > 1:
        y = np.argmax(y, axis=1)

    estimator = KMeans(n_clusters=k)
    ARI_list = []  # adjusted_rand_score(
    NMI_list = []
    AMI_list = []
    if time:
        # print('KMeans exps {}次 æ±~B平å~]~G '.format(time))
        for i in range(time):
            estimator.fit(x, y)
            y_pred = estimator.predict(x)
            score = normalized_mutual_info_score(y, y_pred)
            NMI_list.append(score)
            s2 = adjusted_rand_score(y, y_pred)
            ARI_list.append(s2)
            metric_ami = adjusted_mutual_info_score(y, y_pred)
        # print('NMI_list: {}'.format(NMI_list))
        nmi_score = np.mean(NMI_list)
        ami_score = np.mean(AMI_list)
        ari_score = np.mean(ARI_list)
        print('NMI (10 avg): {:.4f} , AMI (10avg): {:.4f},  ARI (10avg): {:.4f}'.format(nmi_score, ami_score, ari_score))

    else:
        estimator.fit(x, y)
        y_pred = estimator.predict(x)
        score = normalized_mutual_info_score(y, y_pred)
        print("NMI on all label data: {:.5f}".format(score))
    if return_NMI:
        return nmi_score, ami_score, ari_score

# models

## BaseGAttN

In [24]:
var = tf.Variable(np.random.random(size=(1,)))
print(type([var]))

<class 'list'>


In [25]:
import tensorflow as tf

In [26]:
class BaseGAttN:
    def loss(logits, labels, nb_classes, class_weights):
        sample_wts = tf.reduce_sum(tf.multiply(
            tf.one_hot(labels, nb_classes), class_weights), axis=-1)
        xentropy = tf.multiply(tf.nn.sparse_softmax_cross_entropy_with_logits(  # 计算多分类交叉熵
            labels=labels, logits=logits), sample_wts)
        return tf.reduce_mean(xentropy, name='xentropy_mean')
    
    # 更新梯度权重
    def training(loss, lr, l2_coef):
        # weight decay
        vars = tf.compat.v1.trainable_variables()  # 查看可训练变量,list
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in vars if v.name not    # add_n实现列表相加；l2_loss是l2范数值得一半
                           in ['bias', 'gamma', 'b', 'g', 'beta']]) * l2_coef
        # lossL2, tensor(mul_5.0), shape=()
        # optimizer
        opt = tf.compat.v1.train.AdamOptimizer(learning_rate=lr)  # adam函数

        # training op
        train_op = opt.minimize(loss + lossL2)  # 计算梯度，然后更新参数

        return train_op
    
    def preshape(logits, labels, nb_classes):
        new_sh_lab = [-1]
        new_sh_log = [-1, nb_classes]
        log_resh = tf.reshape(logits, new_sh_log)
        lab_resh = tf.reshape(labels, new_sh_lab)
        return log_resh, lab_resh

    def confmat(logits, labels):
        preds = tf.argmax(logits, axis=1)  # 返回行最大值索引
        return tf.confusion_matrix(labels, preds)  # 混淆矩阵
    
    
    ##########################
    # Adapted from tkipf/gcn #
    ##########################

    def masked_softmax_cross_entropy(logits, labels, mask):
        """Softmax cross-entropy loss with masking."""
        loss = tf.nn.softmax_cross_entropy_with_logits(  # 返回交叉熵向量
            logits=logits, labels=labels)
        mask = tf.cast(mask, dtype=tf.float32)  # 改变tensor数据类型
        mask /= tf.reduce_mean(mask)  # 通过均值求loss
        loss *= mask
        return tf.reduce_mean(loss)
    
    def masked_sigmoid_cross_entropy(logits, labels, mask):
        """Softmax cross-entropy loss with masking."""
        labels = tf.cast(labels, dtype=tf.float32)
        loss = tf.nn.sigmoid_cross_entropy_with_logits(  # sigmoid交叉熵
            logits=logits, labels=labels)
        loss = tf.reduce_mean(loss, axis=1)
        mask = tf.cast(mask, dtype=tf.float32)
        mask /= tf.reduce_mean(mask)
        loss *= mask
        return tf.reduce_mean(loss)
    
    def masked_accuracy(logits, labels, mask):
        """Accuracy with masking."""
        correct_prediction = tf.equal(   # 判断向量元素是否相等
            tf.argmax(logits, 1), tf.argmax(labels, 1))
        accuracy_all = tf.cast(correct_prediction, tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        mask /= tf.reduce_mean(mask)
        accuracy_all *= mask
        return tf.reduce_mean(accuracy_all)
    
    def micro_f1(logits, labels, mask):
        """Accuracy with masking."""
        predicted = tf.round(tf.nn.sigmoid(logits))  # 四舍五入函数

        # Use integers to avoid any nasty FP behaviour
        predicted = tf.cast(predicted, dtype=tf.int32)
        labels = tf.cast(labels, dtype=tf.int32)
        mask = tf.cast(mask, dtype=tf.int32)

        # expand the mask so that broadcasting works ([nb_nodes, 1])
        mask = tf.expand_dims(mask, -1)

        # Count true positives, true negatives, false positives and false negatives.
        tp = tf.count_nonzero(predicted * labels * mask)  # 非零元素个数
        tn = tf.count_nonzero((predicted - 1) * (labels - 1) * mask)
        fp = tf.count_nonzero(predicted * (labels - 1) * mask)
        fn = tf.count_nonzero((predicted - 1) * labels * mask)

        # Calculate accuracy, precision, recall and F1 score.
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        fmeasure = (2 * precision * recall) / (precision + recall)
        fmeasure = tf.cast(fmeasure, tf.float32)
        return fmeasure

## GAT

In [27]:
import numpy as np
import tensorflow as tf

### GAT

In [28]:
class GAT(BaseGAttN):
    def inference(inputs, nb_classes, nb_nodes, training, attn_drop, ffd_drop,
                  bias_mat, hid_units, n_heads, activation=tf.nn.elu, residual=False):  # 残差
        attns = []
        for _ in range(n_heads[0]):
            attns.append(attn_head(inputs, bias_mat=bias_mat,
                                          out_sz=hid_units[0], activation=activation,
                                          in_drop=ffd_drop, coef_drop=attn_drop, residual=False))
        h_1 = tf.concat(attns, axis=-1)
        # multi-head attention
        for i in range(1, len(hid_units)):
            h_old = h_1
            attns = []
            for _ in range(n_heads[i]):
                attns.append(attn_head(h_1, bias_mat=bias_mat,
                                              out_sz=hid_units[i], activation=activation,
                                              in_drop=ffd_drop, coef_drop=attn_drop, residual=residual))
            h_1 = tf.concat(attns, axis=-1)
        out = []
        for i in range(n_heads[-1]):
            out.append(attn_head(h_1, bias_mat=bias_mat,
                                        out_sz=nb_classes, activation=lambda x: x,
                                        in_drop=ffd_drop, coef_drop=attn_drop, residual=False))
        logits = tf.add_n(out) / n_heads[-1]

        return logits

### HeteGAT_multi

In [29]:
class HeteGAT_multi(BaseGAttN):
    '''
    # forward；model = HeteGAT_multi
    logits, final_embedding, att_val = model.inference(ftr_in_list,  # list:3, tensor（1， 3025， 1870）
                                                       nb_classes,   # 3
                                                       nb_nodes,     # 3025
                                                       is_train,     # bool
                                                       attn_drop,    # tensor, ()
                                                       ffd_drop,     # tensor, ()
                                                        bias_mat_list=bias_in_list,  # list:2, tensor()
                                                       hid_units=hid_units,   # hid_units:8
                                                       n_heads=n_heads,       # n_heads: [8, 1]
                                                       residual=residual,     # residual: False
                                                       activation=nonlinearity)  # nonlinearity:tf.nn.elu

    '''
    def inference(ftr_in_list, nb_classes, nb_nodes, training, attn_drop, ffd_drop,
                  bias_mat_list, hid_units, n_heads, activation=tf.nn.elu, residual=False,
                  mp_att_size=128):
        embed_list = []
        for features, bias_mat in zip(ftr_in_list, bias_mat_list):
            attns = []
            jhy_embeds = []
            for _ in range(n_heads[0]):   # [8,1]
                # multi-head attention 计算
                attns.append(attn_head(features, bias_mat=bias_mat,
                                              out_sz=hid_units[0], activation=activation,
                                              in_drop=ffd_drop, coef_drop=attn_drop, residual=False))
            h_1 = tf.concat(attns, axis=-1)

            for i in range(1, len(hid_units)):
                h_old = h_1
                attns = []
                for _ in range(n_heads[i]):
                    attns.append(attn_head(h_1, bias_mat=bias_mat,
                                                  out_sz=hid_units[i],
                                                  activation=activation,
                                                  in_drop=ffd_drop,
                                                  coef_drop=attn_drop, residual=residual))
                h_1 = tf.concat(attns, axis=-1)
            embed_list.append(tf.expand_dims(tf.squeeze(h_1), axis=1))  # list:2. 其中每个元素tensor, (3025, 1, 64)

        multi_embed = tf.concat(embed_list, axis=1)   # tensor, (3025, 2, 64)
        # attention输出：tensor(3025, 64)、softmax概率
        final_embed, att_val = SimpleAttLayer(multi_embed, 
                                              mp_att_size,
                                              time_major=False,
                                              return_alphas=True)

        out = []
        for i in range(n_heads[-1]):  # 1
            # 用于添加一个全连接层(input, output) -> (3025, 3)
            out.append(tf.compat.v1.layers.dense(final_embed, nb_classes, activation=None))  
        #     out.append(attn_head(h_1, bias_mat=bias_mat,
        #                                 out_sz=nb_classes, activation=lambda x: x,
        #                                 in_drop=ffd_drop, coef_drop=attn_drop, residual=False))
        logits = tf.add_n(out) / n_heads[-1]  # add_n是列表相加。tensor,(3025, 3)
        # logits_list.append(logits)
        print('de')

        logits = tf.expand_dims(logits, axis=0)  # (1, 3025, 3)
        # attention通过全连接层预测(1, 3025, 3)、attention final_embedding tensor(3025, 64)、attention 概率
        return logits, final_embed, att_val

### HeteGAT_no_coef

In [30]:
class HeteGAT_no_coef(BaseGAttN):
    def inference(ftr_in_list, nb_classes, nb_nodes, is_train, attn_drop, ffd_drop,
                  bias_mat_list, hid_units, n_heads, activation=tf.nn.elu, residual=False,
                  mp_att_size=128):
        embed_list = []
        # coef_list = []
        for bias_mat in bias_mat_list:
            attns = []
            head_coef_list = []
            for _ in range(n_heads[0]):

                attns.append(attn_head(ftr_in_list, bias_mat=bias_mat,
                                                  out_sz=hid_units[0], activation=activation,
                                                  in_drop=ffd_drop, coef_drop=attn_drop, residual=False,
                                                  return_coef=return_coef))
            h_1 = tf.concat(attns, axis=-1)
            for i in range(1, len(hid_units)):
                h_old = h_1
                attns = []
                for _ in range(n_heads[i]):
                    attns.append(attn_head(h_1,
                                                  bias_mat=bias_mat,
                                                  out_sz=hid_units[i],
                                                  activation=activation,
                                                  in_drop=ffd_drop,
                                                  coef_drop=attn_drop,
                                                  residual=residual))
                h_1 = tf.concat(attns, axis=-1)
            embed_list.append(tf.expand_dims(tf.squeeze(h_1), axis=1))
        # att for metapath
        # prepare shape for SimpleAttLayer
        # print('att for mp')
        multi_embed = tf.concat(embed_list, axis=1)
        final_embed, att_val = SimpleAttLayer(multi_embed, mp_att_size,
                                                     time_major=False,
                                                     return_alphas=True)
        # print(att_val)
        # last layer for clf
        out = []
        for i in range(n_heads[-1]):
            out.append(tf.compat.v1.layers.dense(final_embed, nb_classes, activation=None))
        #     out.append(attn_head(h_1, bias_mat=bias_mat,
        #                                 out_sz=nb_classes, activation=lambda x: x,
        #                                 in_drop=ffd_drop, coef_drop=attn_drop, residual=False))
        logits = tf.add_n(out) / n_heads[-1]
        # logits_list.append(logits)
        print('de')
        logits = tf.expand_dims(logits, axis=0)
        # if return_coef:
        #     return logits, final_embed, att_val, coef_list
        # else:
        return logits, final_embed, att_val

### HeteGAT

In [31]:
class HeteGAT(BaseGAttN):
    def inference(inputs, nb_classes, nb_nodes, training, attn_drop, ffd_drop,
                  bias_mat_list, hid_units, n_heads, activation=tf.nn.elu, residual=False,
                  mp_att_size=128,
                  return_coef=False):
        embed_list = []
        coef_list = []
        for bias_mat in bias_mat_list:
            attns = []
            head_coef_list = []
            for _ in range(n_heads[0]):
                if return_coef:
                    a1, a2 = attn_head(inputs, bias_mat=bias_mat,
                                              out_sz=hid_units[0], activation=activation,
                                              in_drop=ffd_drop, coef_drop=attn_drop, residual=False,
                                              return_coef=return_coef)
                    attns.append(a1)
                    head_coef_list.append(a2)
                    # attns.append(attn_head(inputs, bias_mat=bias_mat,
                    #                               out_sz=hid_units[0], activation=activation,
                    #                               in_drop=ffd_drop, coef_drop=attn_drop, residual=False,
                    #                               return_coef=return_coef)[0])
                    #
                    # head_coef_list.append(attn_head(inputs, bias_mat=bias_mat,
                    #                                        out_sz=hid_units[0], activation=activation,
                    #                                        in_drop=ffd_drop, coef_drop=attn_drop,
                    #                                        residual=False,
                    #                                        return_coef=return_coef)[1])
                else:
                    attns.append(attn_head(inputs, bias_mat=bias_mat,
                                                  out_sz=hid_units[0], activation=activation,
                                                  in_drop=ffd_drop, coef_drop=attn_drop, residual=False,
                                                  return_coef=return_coef))
            head_coef = tf.concat(head_coef_list, axis=0)
            head_coef = tf.reduce_mean(head_coef, axis=0)
            coef_list.append(head_coef)
            h_1 = tf.concat(attns, axis=-1)
            for i in range(1, len(hid_units)):
                h_old = h_1
                attns = []
                for _ in range(n_heads[i]):
                    attns.append(attn_head(h_1,
                                                  bias_mat=bias_mat,
                                                  out_sz=hid_units[i],
                                                  activation=activation,
                                                  in_drop=ffd_drop,
                                                  coef_drop=attn_drop,
                                                  residual=residual))
                h_1 = tf.concat(attns, axis=-1)
            embed_list.append(tf.expand_dims(tf.squeeze(h_1), axis=1))
        # att for metapath
        # prepare shape for SimpleAttLayer
        # print('att for mp')
        multi_embed = tf.concat(embed_list, axis=1)
        final_embed, att_val = SimpleAttLayer(multi_embed, mp_att_size,
                                                     time_major=False,
                                                     return_alphas=True)
        # print(att_val)
        # last layer for clf
        out = []
        for i in range(n_heads[-1]):
            out.append(tf.compat.v1.layers.dense(final_embed, nb_classes, activation=None))
        #     out.append(attn_head(h_1, bias_mat=bias_mat,
        #                                 out_sz=nb_classes, activation=lambda x: x,
        #                                 in_drop=ffd_drop, coef_drop=attn_drop, residual=False))
        logits = tf.add_n(out) / n_heads[-1]
        # logits_list.append(logits)
        logits = tf.expand_dims(logits, axis=0)
        if return_coef:
            return logits, final_embed, att_val, coef_list
        else:
            return logits, final_embed, att_val

# Run

In [125]:
import time
import numpy as np
import tensorflow as tf
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3'  # 设置使用GPU1,2,3

In [32]:
config = tf.compat.v1.ConfigProto()  # 用来对session进行参数配置
config.gpu_options.allow_growth = True  # 允许tf自动选择一个存在并且可用的设备来运行操作。

In [33]:
dataset = 'acm'
featype = 'fea'
checkpt_file = os.path.abspath(os.path.dirname(os.getcwd())) +\
                            '/result/Re_HAN_result/offline_result.ckpt'
print('model: {}'.format(checkpt_file))
# training params
batch_size = 1
nb_epochs = 200
patience = 100
lr = 0.05  # learning rate
l2_coef = 0.005  # weight decay
# numbers of hidden units per each attention head in each layer
hid_units = [8]
n_heads = [8, 1]  # additional entry for the output layer
residual = False
nonlinearity = tf.nn.elu
model = HeteGAT_multi

print('Dataset: ' + dataset)
print('----- Opt. hyperparams -----')
print('lr: ' + str(lr))
print('l2_coef: ' + str(l2_coef))
print('----- Archi. hyperparams -----')
print('nb. layers: ' + str(len(hid_units)))
print('nb. units per layer: ' + str(hid_units))
print('nb. attention heads: ' + str(n_heads))
print('residual: ' + str(residual))
print('nonlinearity: ' + str(nonlinearity))
print('model: ' + str(model))

model: D:\PycharmProjects\GNN_Event_Detection_models/result/Re_HAN_result/offline_result.ckpt
Dataset: acm
----- Opt. hyperparams -----
lr: 0.005
l2_coef: 0.001
----- Archi. hyperparams -----
nb. layers: 1
nb. units per layer: [8]
nb. attention heads: [8, 1]
residual: False
nonlinearity: <function elu at 0x000001F86E25EF70>
model: <class '__main__.HeteGAT_multi'>


## jhy data

In [34]:
# jhy data
import scipy.io as sio
import scipy.sparse as sp

### sample_mask

In [35]:
def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool_)

### load_offline_data

In [36]:
from scipy import sparse
import torch

import os
project_path = os.path.abspath(os.path.dirname(os.getcwd()))  # # 获取上级路径

In [37]:
project_path

'D:\\PycharmProjects\\GNN_Event_Detection_models'

In [38]:
load_data_path = project_path + '/result/FinEvent result/offline dataset'
load_embeddings_path = project_path + '/result/FinEvent result/offline result/offline_embeddings'

In [39]:
print(load_data_path)
print(load_embeddings_path)

D:\PycharmProjects\GNN_Event_Detection_models/result/FinEvent result/offline dataset
D:\PycharmProjects\GNN_Event_Detection_models/result/FinEvent result/offline result/offline_embeddings


In [135]:
def load_offline_data(load_data_path, load_embeddings_path):
#     data = sio.loadmat(path)  # load .mat file
    '''
    全是ndarray
    PTP: (3025, 3025)，全是1   <---meta-paths得到的homo 邻接矩阵。
    PLP: (3025, 3025)，有0有1，有些向量时相同的
    PAP: (3025, 3025)，对角线全是1，其他元素基本是0，很少是1.
    feature: (3025, 1870)，由0、1组成。
    label: (3025, 3),就3列，1061、965、999
    train_idx: (1, 600)，0-2225随机抽取的索引
    val_idx: (1, 300)，200-2325之间随机抽取的索引
    test_idx: (1, 2125)，300-3024之间随机抽取的索引
    '''
    
    labels = np.load(load_data_path + '/sorted_labels.npy', allow_pickle=True)
    # 将单行y转换成label矩阵，一列代表one label
    labels_dict = {}
    count = 0
    for i in range(len(labels)):
    #     print(i)
        element = labels[i]
        if element in labels_dict:
            labels[i] = labels_dict[element]
        else:
            labels_dict[element] = count
            labels[i] = count
            count += 1
    truelabels = np.zeros((labels.shape[0], max(labels) + 1))
    for i, label in enumerate(labels):
        print(i, label)
        truelabels[i][label] = 1

    truefeatures = np.load(load_data_path + '/sorted_combined_features_embeddings.npy', allow_pickle=True)
    
    N = truefeatures.shape[0]
    adj_entity = sparse.load_npz(load_data_path + '/s_m_tid_entity_tid_matrix.npz').todense()
    adj_entity = np.asarray(adj_entity) - np.eye(N)
    adj_userid = sparse.load_npz(load_data_path + '/s_m_tid_userid_tid_matrix.npz').todense()
    adj_userid = np.asarray(adj_userid) - np.eye(N)
    rownetworks = [adj_entity, adj_userid]  # , data['PTP'] - np.eye(N)]
    '''
    rownetworks: list: 2。第1个元素，ndarray,(3025, 3025)；第2个元素，ndarray，(3025, 3025)
    '''
    y = truelabels    # shape为(3025, 3)
    train_idx = torch.load(load_embeddings_path + '/block_0/train_mask.pt')  # (1, 600)
    train_idx = np.asarray(torch.unsqueeze(train_idx,0))
    print(train_idx.shape)
    val_idx = torch.load(load_embeddings_path + '/block_0/valid_mask.pt')      # (1, 300)
    val_idx = np.asarray(torch.unsqueeze(val_idx,0))
    test_idx = torch.load(load_embeddings_path + '/block_0/test_mask.pt')    # (1, 2125)
    test_idx = np.asarray(torch.unsqueeze(test_idx,0))

    train_mask = sample_mask(train_idx, y.shape[0])  # 3025长度的bool list，train_idx位置为True
    val_mask = sample_mask(val_idx, y.shape[0])      # 3025长度的boolean list
    test_mask = sample_mask(test_idx, y.shape[0])
    
    # 提取train、val、test的标签
    # 所以，为什么不直接用train_test_split呢？
    y_train = np.zeros(y.shape)  # shape为(3025, 3)的zero 列表
    y_val = np.zeros(y.shape)
    y_test = np.zeros(y.shape)
    y_train[train_mask, :] = y[train_mask, :]  # 取出train_idx为true的label，放入y_train，y_train其余位置为0
    y_val[val_mask, :] = y[val_mask, :]
    y_test[test_mask, :] = y[test_mask, :]

    # return selected_idx, selected_idx_2
    print('y_train:{}, y_val:{}, y_test:{}, train_idx:{}, val_idx:{}, test_idx:{}'.format(y_train.shape,
                                                                                          y_val.shape,
                                                                                          y_test.shape,
                                                                                          train_idx.shape,
                                                                                          val_idx.shape,
                                                                                          test_idx.shape))
    truefeatures_list = [truefeatures, truefeatures, truefeatures]   # truefeatures: (3025, 1870)
    return rownetworks, truefeatures_list, y_train, y_val, y_test, train_mask, val_mask, test_mask

In [136]:
# use adj_list as fea_list, have a try~
adj_list, fea_list, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_offline_data(load_data_path, load_embeddings_path)
if featype == 'adj':
    fea_list = adj_list

(1, 8380)
y_train:(11971, 1), y_val:(11971, 1), y_test:(11971, 1), train_idx:(1, 8380), val_idx:(1, 2394), test_idx:(1, 1197)


In [137]:
fea_list[0].shape

(11971, 302)

In [138]:
fea_list

[array([[-1.52512252e+00, -1.28044748e+00, -1.67360997e+00, ...,
         -1.10073507e+00,  4.11920000e+00,  1.50462963e-04],
        [-8.79233256e-02, -6.25963390e-01, -9.90003288e-01, ...,
          4.68896657e-01,  4.11920000e+00,  4.97685185e-04],
        [-7.38342285e-01, -2.31036329e+00, -1.53008020e+00, ...,
          3.20886701e-01,  4.11920000e+00,  5.78703704e-04],
        ...,
        [ 8.28718364e-01,  1.84172881e+00, -1.79105675e+00, ...,
          7.84933940e-02,  4.11950000e+00,  9.96261574e-01],
        [-3.74932468e-01, -4.66887876e-02, -1.77780282e+00, ...,
          6.00812554e-01,  4.11950000e+00,  9.97430556e-01],
        [-3.72165012e+00, -1.18709993e+00,  3.44226480e+00, ...,
         -4.23700523e+00,  4.11950000e+00,  9.99652778e-01]]),
 array([[-1.52512252e+00, -1.28044748e+00, -1.67360997e+00, ...,
         -1.10073507e+00,  4.11920000e+00,  1.50462963e-04],
        [-8.79233256e-02, -6.25963390e-01, -9.90003288e-01, ...,
          4.68896657e-01,  4.11920000e

In [139]:
train_mask

array([ True,  True,  True, ..., False,  True, False])

In [140]:
y_train

array([[394.],
       [394.],
       [394.],
       ...,
       [  0.],
       [279.],
       [  0.]])

## run_model

In [141]:
import scipy.sparse as sp

### add_data_dimension

In [143]:
# truefeatures: (3025, 1870)
nb_nodes = fea_list[0].shape[0]  # 3025
ft_size = fea_list[0].shape[1]   # 1870
nb_classes = y_train.shape[1]    # 3

# adj = adj.todense()

# features = features[np.newaxis]  # [1, nb_node, ft_size]
fea_list = [fea[np.newaxis] for fea in fea_list]  # np.newaxis行增加一个新的维度
'''
fea_list:list:3。第一个元素，ndarray, (1, 3025, 1870)
'''
adj_list = [adj[np.newaxis] for adj in adj_list]  # adj_list: 2. 单个元素(1, 3025, 3025)
y_train = y_train[np.newaxis]  # ndarray: (1, 3025, 3)
y_val = y_val[np.newaxis]      # ndarray: (1, 3025, 3)
y_test = y_test[np.newaxis]    # ndarray: (1, 3025, 3)
train_mask = train_mask[np.newaxis]  # ndarray(1, 3025)
val_mask = val_mask[np.newaxis]      # ndarray(1, 3025)
test_mask = test_mask[np.newaxis]    # ndarray(1, 3025)

biases_list = [adj_to_bias(adj, [nb_nodes], nhood=1) for adj in adj_list]

In [144]:
fea_list[0].shape

(1, 11971, 302)

### build graph

In [145]:
print('build graph...')
with tf.Graph().as_default():  # 创建一个新的计算图
    with tf.name_scope('input'):  # 创建一个上下文管理器
        ftr_in_list = [tf.compat.v1.placeholder(dtype=tf.float32,  # 占位符，提前分配必要的内存
                                      shape=(batch_size, nb_nodes, ft_size),  # batch_size:1, nb_nodes:3025, fea_size: 1870
                                      name='ftr_in_{}'.format(i))
                       for i in range(len(fea_list))]  # fea_list,长度为3，内部单个元素，(1, 3025, 1870)
        bias_in_list = [tf.compat.v1.placeholder(dtype=tf.float32,
                                       shape=(batch_size, nb_nodes, nb_nodes),
                                       name='bias_in_{}'.format(i))
                        for i in range(len(biases_list))]  # 邻接矩阵转换成的biases_list: 2. 单个元素占位符tensor, (1, 3025, 3025)
        lbl_in = tf.compat.v1.placeholder(dtype=tf.int32, 
                                        shape=(batch_size, nb_nodes, nb_classes),   # tensor, nb_classes: 3
                                        name='lbl_in')   
        msk_in = tf.compat.v1.placeholder(dtype=tf.int32, 
                                        shape=(batch_size, nb_nodes),  # tensor, (1, 3025)
                                        name='msk_in')
        attn_drop = tf.compat.v1.placeholder(dtype=tf.float32, shape=(), name='attn_drop')  # tensor, ()
        ffd_drop = tf.compat.v1.placeholder(dtype=tf.float32, shape=(), name='ffd_drop')  # # tensor, ()
        is_train = tf.compat.v1.placeholder(dtype=tf.bool, shape=(), name='is_train')  # # tensor, ()
    # forward；model = HeteGAT_multi
    logits, final_embedding, att_val = model.inference(ftr_in_list,  # list:3, tensor（1， 3025， 1870）
                                                       nb_classes,   # 3
                                                       nb_nodes,     # 3025
                                                       is_train,     # bool
                                                       attn_drop,    # tensor, ()
                                                       ffd_drop,     # tensor, ()
                                                       bias_mat_list=bias_in_list,  # list:2, tensor(1, 3025, 3025)
                                                       hid_units=hid_units,   # hid_units:[8]
                                                       n_heads=n_heads,       # n_heads: [8, 1]
                                                       residual=residual,     # residual: False
                                                       activation=nonlinearity)  # nonlinearity:tf.nn.elu

    # cal masked_loss
    log_resh = tf.reshape(logits, [-1, nb_classes])  # （3025， 3）
    lab_resh = tf.reshape(lbl_in, [-1, nb_classes])  # （3025， 3）
    msk_resh = tf.reshape(msk_in, [-1])              # mask，（3025， ）
    loss = model.masked_softmax_cross_entropy(log_resh, lab_resh, msk_resh)  # 占位符计算softmax cross_entropy based on (pred, y)
    accuracy = model.masked_accuracy(log_resh, lab_resh, msk_resh)           # 计算accuracy
    # optimzie
    train_op = model.training(loss, lr, l2_coef)  # lr = 0.005、l2_coef = 0.001

    saver = tf.compat.v1.train.Saver()  # 用于保存模型

    init_op = tf.group(tf.compat.v1.global_variables_initializer(),  # 全局变量初始化；group组合多个operation
                       tf.compat.v1.local_variables_initializer())

    vlss_mn = np.inf
    vacc_mx = 0.0
    curr_step = 0

    with tf.compat.v1.Session(config=config) as sess:  # 创建session
        sess.run(init_op)

        train_loss_avg = 0
        train_acc_avg = 0
        val_loss_avg = 0
        val_acc_avg = 0

        for epoch in range(nb_epochs):  # 200
            tr_step = 0
           
            tr_size = fea_list[0].shape[0]
            # ================   training    ============
            while tr_step * batch_size < tr_size:
                # feature,占位符内存已经分配完毕，fea_list是真实数据，输入进行训练模型
                fd1 = {i: d[tr_step * batch_size:(tr_step + 1) * batch_size]  # dict:3. 每个元素tensor, (1, 3025, 1870)
                       for i, d in zip(ftr_in_list, fea_list)}
                # bias
                fd2 = {i: d[tr_step * batch_size:(tr_step + 1) * batch_size]  # dict: 2. 每个元素tensor, (1, 3025, 3025)
                       for i, d in zip(bias_in_list, biases_list)}
                # other params
                fd3 = {lbl_in: y_train[tr_step * batch_size:(tr_step + 1) * batch_size],
                       msk_in: train_mask[tr_step * batch_size:(tr_step + 1) * batch_size],
                       is_train: True,
                       attn_drop: 0.6,
                       ffd_drop: 0.6}
                fd = fd1
                fd.update(fd2)  # 字典update方法
                fd.update(fd3)  # 获得字典形式的所有数据、参数
                # training操作：更新权重；计算loss；计算accuracy；attention概率
                _, loss_value_tr, acc_tr, att_val_train = sess.run([train_op, loss, accuracy, att_val],
                                                                   feed_dict=fd)
                train_loss_avg += loss_value_tr
                train_acc_avg += acc_tr
                tr_step += 1

            vl_step = 0
            vl_size = fea_list[0].shape[0]
            # =============   val       =================
            while vl_step * batch_size < vl_size:
                # fd1 = {ftr_in: features[vl_step * batch_size:(vl_step + 1) * batch_size]}
                fd1 = {i: d[vl_step * batch_size:(vl_step + 1) * batch_size]
                       for i, d in zip(ftr_in_list, fea_list)}
                fd2 = {i: d[vl_step * batch_size:(vl_step + 1) * batch_size]
                       for i, d in zip(bias_in_list, biases_list)}
                fd3 = {lbl_in: y_val[vl_step * batch_size:(vl_step + 1) * batch_size],
                       msk_in: val_mask[vl_step * batch_size:(vl_step + 1) * batch_size],
                       is_train: False,
                       attn_drop: 0.0,
                       ffd_drop: 0.0}
          
                fd = fd1
                fd.update(fd2)
                fd.update(fd3)
                loss_value_vl, acc_vl = sess.run([loss, accuracy],
                                                 feed_dict=fd)
                val_loss_avg += loss_value_vl
                val_acc_avg += acc_vl
                vl_step += 1
            # import pdb; pdb.set_trace()
            print('Epoch: {}, att_val: {}'.format(epoch, np.mean(att_val_train, axis=0)))
            print('Training: loss = %.5f, acc = %.5f | Val: loss = %.5f, acc = %.5f' %
                  (train_loss_avg / tr_step, train_acc_avg / tr_step,
                   val_loss_avg / vl_step, val_acc_avg / vl_step))
            
            # =============   judging  =================
            if val_acc_avg / vl_step >= vacc_mx or val_loss_avg / vl_step <= vlss_mn:
                if val_acc_avg / vl_step >= vacc_mx and val_loss_avg / vl_step <= vlss_mn:
                    vacc_early_model = val_acc_avg / vl_step
                    vlss_early_model = val_loss_avg / vl_step
                    saver.save(sess, checkpt_file)
                vacc_mx = np.max((val_acc_avg / vl_step, vacc_mx))
                vlss_mn = np.min((val_loss_avg / vl_step, vlss_mn))
                curr_step = 0
            else:
                curr_step += 1
                if curr_step == patience:
                    print('Early stop! Min loss: ', vlss_mn,
                          ', Max accuracy: ', vacc_mx)
                    print('Early stop model validation loss: ',
                          vlss_early_model, ', accuracy: ', vacc_early_model)
                    break

            train_loss_avg = 0
            train_acc_avg = 0
            val_loss_avg = 0
            val_acc_avg = 0
        
        # loading model params
        saver.restore(sess, checkpt_file)
        print('load model from : {}'.format(checkpt_file))
        ts_size = fea_list[0].shape[0]
        ts_step = 0
        ts_loss = 0.0
        ts_acc = 0.0
        
        # ============= testing =================
        while ts_step * batch_size < ts_size:
            # fd1 = {ftr_in: features[ts_step * batch_size:(ts_step + 1) * batch_size]}
            fd1 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                   for i, d in zip(ftr_in_list, fea_list)}
            fd2 = {i: d[ts_step * batch_size:(ts_step + 1) * batch_size]
                   for i, d in zip(bias_in_list, biases_list)}
            fd3 = {lbl_in: y_test[ts_step * batch_size:(ts_step + 1) * batch_size],
                   msk_in: test_mask[ts_step * batch_size:(ts_step + 1) * batch_size],
            
                   is_train: False,
                   attn_drop: 0.0,
                   ffd_drop: 0.0}
        
            fd = fd1
            fd.update(fd2)
            fd.update(fd3)
            loss_value_ts, acc_ts, jhy_final_embedding = sess.run([loss, accuracy, final_embedding],
                                                                  feed_dict=fd)
            ts_loss += loss_value_ts
            ts_acc += acc_ts
            ts_step += 1

        print('Test loss:', ts_loss / ts_step,
              '; Test accuracy:', ts_acc / ts_step)

        print('start knn, kmean.....')
        xx = np.expand_dims(jhy_final_embedding, axis=0)[test_mask]
  
        from numpy import linalg as LA

        # xx = xx / LA.norm(xx, axis=1)
        yy = y_test[test_mask]

        print('xx: {}, yy: {}'.format(xx.shape, yy.shape))

#         my_KNN(xx, yy)
        my_Kmeans(xx, yy)

        sess.close()

build graph...


C:\Users\yysgz\AppData\Local\Temp\ipykernel_836\3938853343.py:20: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
  features_fts = tf.compat.v1.layers.conv1d(features, out_sz, 1, use_bias=False)  # 一维卷积操作, out: (1, 3025, 8)
C:\Users\yysgz\AppData\Local\Temp\ipykernel_836\3938853343.py:22: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
  f_1 = tf.compat.v1.layers.conv1d(features_fts, 1, 1)  # (1, 3025, 1)
C:\Users\yysgz\AppData\Local\Temp\ipykernel_836\3938853343.py:23: UserWarning: `tf.layers.conv1d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv1D` instead.
  f_2 = tf.compat.v1.layers.conv1d(features_fts, 1, 1)  # (1, 3025, 1)


ValueError: Dimensions must be equal, but are 64 and 11971 for '{{node v/Tensordot/MatMul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false](v/Tensordot/Reshape, v/Tensordot/ReadVariableOp)' with input shapes: [2394200,64], [11971,128].

# baselines

## load_tweeter_data

In [4]:
import numpy as np
import pandas as pd

import os
project_path = os.path.abspath(os.path.dirname(os.getcwd()))  # # 获取上级路径

load_path = project_path + '/data/FinEvent_datasets/raw dataset/'
save_path = project_path + '/result/FinEvent result/'

In [5]:
print(load_path)
print(save_path)

D:\PycharmProjects\GNN_Event_Detection_models/data/FinEvent_datasets/raw dataset/
D:\PycharmProjects\GNN_Event_Detection_models/result/FinEvent result/


In [6]:
import datetime

# load data (68841 tweets, multiclasses filtered)
p_part1 = load_path + '68841_tweets_multiclasses_filtered_0722_part1.npy'
p_part2 = load_path + '68841_tweets_multiclasses_filtered_0722_part2.npy'
# allow_pickle: 可选，布尔值，允许使用 Python pickles 保存对象数组，Python 中的 pickle 用于在保存到磁盘文件或从磁盘文件读取之前，对对象进行序列化和反序列化。
np_part1 = np.load(p_part1, allow_pickle=True)   # (35000, 16)
np_part2 = np.load(p_part2, allow_pickle=True)   # (33841, 16)

np_tweets = np.concatenate((np_part1, np_part2), axis=0)  # (68841, 16)
print('Data loaded.')

df = pd.DataFrame(data=np_tweets, columns=['event_id', 'tweet_id', 'text', 'user_id', 'created_at', 'user_loc',
                                      'place_type', 'place_full_name', 'place_country_code', 'hashtags',
                                      'user_mentions', 'image_urls', 'entities', 'words', 'filtered_words', 'sampled_words'])
print('Data converted to dataframe.')
# sort date by time
df = df.sort_values(by='created_at').reset_index(drop=True)

# append date
df['date'] = [d.date() for d in df['created_at']]
# 因为graph太大，爆了内存，所以取4天的twitter data做demo，后面用nci server
init_day = df.loc[0, 'date']
df = df[(df['date']>= init_day) & (df['date']<= init_day + datetime.timedelta(days=3))].reset_index() # (11971, 18)
print(df.shape)
print(df.event_id.nunique())
print(df.user_id.nunique())

Data loaded.
Data converted to dataframe.
(11971, 18)
89
10905


In [7]:
df.shape

(11971, 18)

## Bert

### bert_model_example

In [8]:
from transformers import AutoTokenizer, AutoModel

from transformers import logging
logging.set_verbosity_warning()

In [9]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model = AutoModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
inputs = tokenizer('hello world', return_tensors='pt')
outputs = bert_model(**inputs)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [11]:
outputs[0][0][0][:100].shape  # 取第1层，也可以取别的层。

torch.Size([100])

In [12]:
df.head()

,index,event_id,tweet_id,text,user_id,created_at,user_loc,place_type,place_full_name,place_country_code,hashtags,user_mentions,image_urls,entities,words,filtered_words,sampled_words,date
0,0,394,255819992157786112,HipHop awards bout to be live!!,250870763,2012-10-10 00:00:13,,,,,[],[],[],[],"[award, live, bout, hiphop]","[award, live, bout, hiphop]",[],2012-10-10
1,1,394,255820118095978496,HIPHOP AWARDS TIME!,28026779,2012-10-10 00:00:43,SoundCloud/RaRaSupaStar,,,,[],[],[],[],"[HIPHOP, AWARDS, time]","[hiphop, awards, time]",[],2012-10-10
2,2,394,255820147489636353,Bet hiphop awards,566825483,2012-10-10 00:00:50,,,,,[],[],[],"[(Bet, GPE)]","[award, bet, hiphop]","[award, bet, hiphop]",[],2012-10-10
3,3,394,255820164023595008,BET HipHop awards is on!!!,197834311,2012-10-10 00:00:54,Saint Lucia ☀️🌴🇱🇨,,,,[],[],[],[],"[HipHop, BET, award]","[hiphop, bet, award]",[],2012-10-10
4,4,394,255820180884701184,Watchin Da BET Hiphop Awards,439490861,2012-10-10 00:00:58,"Michigan, USA",,,,[],[],[],[],"[Hiphop, Watchin, Awards, Da, BET]","[hiphop, watchin, awards, da, bet]",[],2012-10-10


### bert_embeddings

In [21]:
df.shape

(11971, 18)

In [136]:
# 文件太大，爆内存df['bert_embeddings'] = df.text.apply(lambda x: bert_model(**tokenizer(x, return_tensors='pt'))[0])
# solution: mini-batch
import torch

from sklearn.model_selection import train_test_split
import math
from sklearn.cluster import DBSCAN,KMeans
# NMI, AMI, ARI
from sklearn.metrics import normalized_mutual_info_score, adjusted_mutual_info_score, adjusted_rand_score

# 生成随机序列
random_seq = torch.randperm(df.shape[0])

# tran_indies, test_indies = train_test_split(random_seq, test_size=0.2)
total_nmi = []
total_ami = []
total_ari = []
batch_size = 100
batch_nums = int(df.shape[0] / batch_size)
# 因为聚类指标无法训练，只能fit和fit_predict()，所以取mini-batch指标平均值
for i in range(batch_nums):
    print(i)
    if i == batch_nums-1:
        mini_embed_indies = random_seq[i*batch_size:]
    else:
        mini_embed_indies = random_seq[i*batch_size:(i+1)*batch_size]
    df_mini_words = df.loc[embed_indies, 'filtered_words'].reset_index()
    embedding_lists = []
    for j in range(len(df_mini_words)):
        inputs = df_mini_words.loc[j,'filtered_words']
        inputs_sentence = ' '.join(inputs)
        inputs_embedding = bert_model(**tokenizer(inputs_sentence, return_tensors='pt'))[0][0][0]
        embedding_lists.append(inputs_embedding.detach().numpy())
#         data_mini_x = torch.cat((data_mini_x, inputs_embedding), dim=0)

#     data_mini_x = [bert_model(**tokenizer(' '.join(i), return_tensors='pt'))[0][0][0]
#                    for i in df_mini_words]
#     data_mini_x = torch.unsqueeze(data_mini_x,1)
#     break
    data_mini_x = np.array(embedding_lists)
    print(type(data_mini_x))
    print(data_mini_x.shape)

    data_mini_y = np.array(df.loc[embed_indies, 'event_id'])
#     data_mini_y = np.expand_dims(data_mini_y, 1)
    print(data_mini_y.shape)
    # 拟合模型
    n_classes = len(np.unique(data_mini_y))
    kmeans_model = KMeans(n_clusters=n_classes).fit(data_mini_x, data_mini_y)  # 聚类模型
#     pred_y = kmeans_model.labels_
    pred_y = kmeans_model.predict(data_mini_x)
    # 计算clustering evaluation指标
    batch_nmi = normalized_mutual_info_score(data_mini_y, pred_y)
    batch_ami = adjusted_mutual_info_score(data_mini_y, pred_y)
    batch_ari = adjusted_rand_score(data_mini_y, pred_y)
    
    total_nmi.append(batch_nmi)
    total_ami.append(batch_ami)
    total_ari.append(batch_ari)

metric_nmi = np.mean(total_nmi
metric_ami = np.mean(total_ami)
metric_ari = np.mean(total_ari)
print('metric_nmi:' + metric_nmi, 'metric_ami:' + metric_ami, 'metric_ari:' + metric_ari)
print('embeddings finished.')

0
<class 'numpy.ndarray'>
(100, 768)
(100,)
1
<class 'numpy.ndarray'>
(100, 768)
(100,)
2
<class 'numpy.ndarray'>
(100, 768)
(100,)
3
<class 'numpy.ndarray'>
(100, 768)
(100,)
4
<class 'numpy.ndarray'>
(100, 768)
(100,)
5
<class 'numpy.ndarray'>
(100, 768)
(100,)
6
<class 'numpy.ndarray'>
(100, 768)
(100,)
7
<class 'numpy.ndarray'>
(100, 768)
(100,)
8
<class 'numpy.ndarray'>
(100, 768)
(100,)
9
<class 'numpy.ndarray'>
(100, 768)
(100,)
10
<class 'numpy.ndarray'>
(100, 768)
(100,)
11
<class 'numpy.ndarray'>
(100, 768)
(100,)
12
<class 'numpy.ndarray'>
(100, 768)
(100,)
13
<class 'numpy.ndarray'>
(100, 768)
(100,)
14
<class 'numpy.ndarray'>
(100, 768)
(100,)
15
<class 'numpy.ndarray'>
(100, 768)
(100,)
16
<class 'numpy.ndarray'>
(100, 768)
(100,)
17
<class 'numpy.ndarray'>
(100, 768)
(100,)
18
<class 'numpy.ndarray'>
(100, 768)
(100,)
19
<class 'numpy.ndarray'>
(100, 768)
(100,)
20
<class 'numpy.ndarray'>
(100, 768)
(100,)
21
<class 'numpy.ndarray'>
(100, 768)
(100,)
22
<class 'numpy.nda

AttributeError: 'list' object has no attribute 'mean'

In [142]:
metric_nmi = np.mean(total_nmi)
metric_ami = np.mean(total_ami)
metric_ari = np.mean(total_ari)

In [143]:
metric_nmi

0.7517083348140321

In [144]:
metric_ami

0.2645850173310259

In [145]:
metric_ari

0.1392537309157387

In [131]:
data_mini_y.shape

(100,)

In [134]:
len(n_classes)

44

In [133]:
data_mini_y

array([240, 1, 279, 419, 502, 91, 78, 1, 278, 18, 91, 2, 1, 1, 1, 282, 91,
       243, 1, 1, 1, 18, 21, 487, 1, 1, 427, 3, 279, 0, 384, 472, 477,
       487, 1, 421, 243, 88, 246, 3, 243, 278, 75, 1, 394, 1, 386, 242, 2,
       75, 502, 241, 243, 19, 422, 419, 1, 2, 246, 475, 18, 280, 21, 83,
       1, 1, 106, 1, 504, 474, 3, 3, 240, 238, 240, 2, 421, 383, 1, 80,
       423, 1, 94, 246, 19, 3, 85, 1, 278, 1, 502, 422, 429, 18, 2, 475,
       279, 425, 1, 3], dtype=object)

In [116]:

x = np.array([[1, 2], [1.5, 1.8], [5, 8], [8, 8], [1, 0.6], [9, 11]])
k_means = KMeans(2)
k_means.fit(x)


KMeans(n_clusters=2)

In [122]:
x

array([[ 1. ,  2. ],
       [ 1.5,  1.8],
       [ 5. ,  8. ],
       [ 8. ,  8. ],
       [ 1. ,  0.6],
       [ 9. , 11. ]])

In [117]:
x.shape

(6, 2)

In [118]:
type(x)

numpy.ndarray

In [106]:
data_mini_x

array([[-0.28958654, -0.12949194,  0.00678096, ..., -0.07712209,
         0.05935615,  0.4994283 ],
       [-0.24104664, -0.00732547,  0.04749957, ..., -0.20405638,
         0.26195958,  0.16743796],
       [-0.3300131 , -0.00179507, -0.3481554 , ..., -0.05225424,
         0.32856098,  0.43455335],
       ...,
       [-0.50517625,  0.07986608, -0.5771193 , ..., -0.47181144,
         0.09768105,  0.49552682],
       [-0.14235994,  0.09129456, -0.01598077, ..., -0.02290452,
         0.27011776,  0.2527727 ],
       [-0.03415905,  0.35531443, -0.10913993, ..., -0.22343796,
         0.11775812,  0.16309094]], dtype=float32)